<h1 align="center">SimpleITK Images and Resampling</h1>


**Summary:**    

1. Images occupy a region in physical space which is defined by:
  * Origin.
  * Size (number of pixels per dimension).
  * Spacing (unknown consistent units: nm, mm, m, km...).
  * Direction cosine matrix (axis directions in physical space).
  
  These attributes are the image's meta-data. Computing the physical coordinates from image indexes requires all four   components.

2. An image may contain a meta-data dictionary. This supplemental information often includes the image modality (e.g. CT), patient name, and information with respect to the image acquisition. 
3. Image initialization: user specified pixel type, user specified dimensionality (2,3), origin at zero, unit spacing in all dimensions and identity direction cosine matrix, intensities set to zero.
4. Data transfer to/from numpy: GetArrayFromImage (copy), GetArrayViewFromImage (immutable), GetImageFromArray (copy) + set the meta-data yourself. 
5. A common issue with resampling resulting in an all black image is due to (a) incorrect specification of the 
desired output image's spatial domain (its meta-data); or (b) the use of the inverse of the transformation mapping from the output spatial domain to the resampled image.

## Images are Physical Objects

<img src="../figures/ImageOriginAndSpacing.png" style="width:700px"/><br><br>

### Pixel Types

The pixel type is represented as an enumerated type. The following is a table of the enumerated list.

<table>
  <tr><td>sitkUInt8</td><td>Unsigned 8 bit integer</td></tr>
  <tr><td>sitkInt8</td><td>Signed 8 bit integer</td></tr>
  <tr><td>sitkUInt16</td><td>Unsigned 16 bit integer</td></tr>
  <tr><td>sitkInt16</td><td>Signed 16 bit integer</td></tr>
  <tr><td>sitkUInt32</td><td>Unsigned 32 bit integer</td></tr>
  <tr><td>sitkInt32</td><td>Signed 32 bit integer</td></tr>
  <tr><td>sitkUInt64</td><td>Unsigned 64 bit integer</td></tr>
  <tr><td>sitkInt64</td><td>Signed 64 bit integer</td></tr>
  <tr><td>sitkFloat32</td><td>32 bit float</td></tr>
  <tr><td>sitkFloat64</td><td>64 bit float</td></tr>
  <tr><td>sitkComplexFloat32</td><td>complex number of 32 bit float</td></tr>
  <tr><td>sitkComplexFloat64</td><td>complex number of 64 bit float</td></tr>
  <tr><td>sitkVectorUInt8</td><td>Multi-component of unsigned 8 bit integer</td></tr>
  <tr><td>sitkVectorInt8</td><td>Multi-component of signed 8 bit integer</td></tr>
  <tr><td>sitkVectorUInt16</td><td>Multi-component of unsigned 16 bit integer</td></tr>
  <tr><td>sitkVectorInt16</td><td>Multi-component of signed 16 bit integer</td></tr>
  <tr><td>sitkVectorUInt32</td><td>Multi-component of unsigned 32 bit integer</td></tr>
  <tr><td>sitkVectorInt32</td><td>Multi-component of signed 32 bit integer</td></tr>
  <tr><td>sitkVectorUInt64</td><td>Multi-component of unsigned 64 bit integer</td></tr>
  <tr><td>sitkVectorInt64</td><td>Multi-component of signed 64 bit integer</td></tr>
  <tr><td>sitkVectorFloat32</td><td>Multi-component of 32 bit float</td></tr>
  <tr><td>sitkVectorFloat64</td><td>Multi-component of 64 bit float</td></tr>
  <tr><td>sitkLabelUInt8</td><td>RLE label of unsigned 8 bit integers</td></tr>
  <tr><td>sitkLabelUInt16</td><td>RLE label of unsigned 16 bit integers</td></tr>
  <tr><td>sitkLabelUInt32</td><td>RLE label of unsigned 32 bit integers</td></tr>
  <tr><td>sitkLabelUInt64</td><td>RLE label of unsigned 64 bit integers</td></tr>
</table>

There is also `sitkUnknown`, which is used for undefined or erroneous pixel ID's.

Some filters only work with images with a specific pixel type. The primary example is the registration framework which works with sitkFloat32 or sitkFloat64. To address this issue you can either specify the appropriate pixel type when reading or creating the image, or use the <a href="https://itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html#af8c9d7cc96a299a05890e9c3db911885">Cast function</a>.  

In [ ]:
library(SimpleITK)

source("downloaddata.R")

OUTPUT_DIR <- 'output'

## Image Creation

The following components are required for a complete definition of an image:
<ol>
<li>Pixel type [fixed on creation, no default]: unsigned 32 bit integer, sitkVectorUInt8, etc., see list above.</li>
<li> Sizes [fixed on creation, no default]: number of pixels/voxels in each dimension. This quantity implicitly defines the image dimension.</li>
<li> Origin [default is zero]: coordinates of the pixel/voxel with index (0,0,0) in physical units (i.e. mm).</li>
<li> Spacing [default is one]: Distance between adjacent pixels/voxels in each dimension given in physical units.</li>
<li> Direction matrix [default is identity]: mapping, rotation, between direction of the pixel/voxel axes and physical directions.</li>
</ol>

Initial pixel/voxel values are set to zero.

In [ ]:
image_3D <- Image(256, 128, 64, "sitkInt16")
image_2D <- Image(64, 64, "sitkFloat32")
image_RGB <- Image(c(128,64), "sitkVectorUInt8", 3)

Show(image_3D)
Show(image_RGB)

Or, creation from file.

In [ ]:
logo <- ReadImage(fetch_data('SimpleITK.jpg'))
Show(logo)

## Basic Image Attributes (Meta-Data)

You can change the image origin, spacing and direction. Making such changes to an image already containing data should be done cautiously. 

In [ ]:
selected_image <- image_3D
cat('Before modification:\n')
cat('origin: ', selected_image$GetOrigin(), "\n")
cat('size: ', selected_image$GetSize(), "\n")
cat('spacing: ', selected_image$GetSpacing(), "\n")
cat('direction: ', selected_image$GetDirection(), "\n")
cat('pixel type: ', selected_image$GetPixelIDTypeAsString(), "\n")
cat('number of pixel components: ', selected_image$GetNumberOfComponentsPerPixel(), "\n")

selected_image$SetOrigin(c(78.0, 76.0, 77.0))
selected_image$SetSpacing(c(0.5,0.5,3.0))

cat('\nAfter modification:', "\n")
cat('origin: ', selected_image$GetOrigin(), "\n")
cat('spacing: ', selected_image$GetSpacing(), "\n")

## Accessing Pixels and Slicing

Either use the ``GetPixel`` and ``SetPixel`` functions or the R slicing operator. The access functions and image slicing operator are in [x,y,z] order. Note that the access functions are zero-based and the slicing operator is one-based.

In [ ]:
# Use access function and slicing operator to read pixel value.
cat(image_3D$GetPixel(c(0, 0, 0)), "\n")
cat(image_3D[1,1,1], "\n")
# Change value using access function.
image_3D$SetPixel(c(0, 0, 0), 1)

cat(image_3D$GetPixel(c(0, 0, 0)), "\n")
cat(image_3D[1,1,1], "\n")

In [ ]:
# Brute force sub-sampling 
logo_subsampled <- logo[seq(1,logo$GetWidth(),2), seq(1,logo$GetHeight(),2)]
Show(logo_subsampled)

# Get the sub-image containing the word Simple
simple <- logo[1:115,]
Show(simple)

# Get the sub-image containing the word Simple and flip it
simple_flipped <- logo[seq(115,1,-1),]
Show(simple_flipped)

## Image operations

SimpleITK supports basic arithmetic operations between images while taking into account their meta-data. Images must physically overlap (pixel by pixel).

How close do physical attributes (meta-data values) need to be in order to be considered equivalent?

In [ ]:
img_width <- 128
img_height <- 64
img1 <- Image(c(img_width, img_height), "sitkUInt8")
invisible(lapply(seq(0,img_width-1,1), function(i) {img1$SetPixel(c(i,1),5)}))

img2 <- Image(img1$GetSize(), img1$GetPixelID())
#img2$SetDirection(c(0,1,0.5,0.5))
img2$SetOrigin(c(0.000001,0.000001))
invisible(lapply(seq(0,img_width-1,1), function(i) {
                                       img2$SetPixel(c(i,1),120) 
                                       img2$SetPixel(c(i,as.integer(img_height/2)),60)}))

img3 <- img1 + img2

Comparative operators (&gt;, &gt;=, &lt;, &lt;=, ==) are also supported, returning binary images.

In [ ]:
thresholded_image <- img3>50
Show(thresholded_image)

## SimpleITK2R and R2SimpleITK


### SimpleITK2R

1. ```as.array()```: returns a copy of the image data. You can then freely modify the data as it has no effect on the original SimpleITK image.

### R2SimpleITK
1. ```as.image()```: returns a SimpleITK image with origin set to zero, spacing set to one for all dimensions, and the direction cosine matrix set to identity. Intensity data is copied from the R array. __In most cases you will need to set appropriate meta-data values.__ 


In [ ]:
arr <- as.array(image_3D)
cat(image_3D$GetSize(), "\n")
cat(dim(arr), "\n")

arr <- as.array(image_RGB)
cat(image_RGB$GetSize(), "\n")
cat(dim(arr), "\n")

In [ ]:
arr <- array(0, dim=c(10,20,3))

img <- as.image(arr)
cat(img$GetSize(), "\n")

## Reading and Writing

SimpleITK can read and write images stored in a single file, or a set of files (e.g. DICOM series). The toolkit provides both an object oriented and a procedural interface. The primary difference being that the object oriented approach provides more control while the procedural interface is more convenient.

We look at DICOM images as an example illustrating this difference. Images stored in the DICOM format have a meta-data dictionary associated with them, which is populated with the DICOM tags. When a DICOM image series is read as a single image volume, the resulting image's meta-data dictionary is not populated since DICOM tags are specific to each of the files in the series. If you use the procedural method for reading the series then you do not have access to the set of meta-data dictionaries associated with each of the files. To obtain each dictionary you will have to access each of the files separately. On the other hand, if you use the object oriented interface, the set of dictionaries will be accessible from the ```ImageSeriesReader``` which you used to read the DICOM series. The meta-data dictionary for each file is available using the <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ImageSeriesReader.html#a337b19b6bc101f5571455afb46514b6d">HasMetaDataKey</a> and <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ImageSeriesReader.html#a19995f33b86c60e2ae4878cb4d8c30ee">GetMetaData</a> methods. 

We start with reading and writing an image using the procedural interface.

In [ ]:
img <- ReadImage(fetch_data('SimpleITK.jpg'))
WriteImage(img, file.path(OUTPUT_DIR, 'SimpleITK.png'))

Read an image in JPEG format and cast the pixel type according to user selection.

In [ ]:
pixel_types <- list("sitkVectorUInt8", "sitkVectorUInt16", "sitkVectorFloat64")

img <- ReadImage(fetch_data('SimpleITK.jpg'), pixel_types[[1]])

Read a DICOM series and write it as a single mha file.

In [ ]:
data_directory <- dirname(fetch_data("CIRS057A_MR_CT_DICOM/readme.txt"))

series_ID <- '1.2.840.113619.2.290.3.3233817346.783.1399004564.515'

# Use the functional interface to read the image series.
original_image <- ReadImage(ImageSeriesReader_GetGDCMSeriesFileNames(data_directory,series_ID))

# Write the image.
output_file_name_3D <- file.path(OUTPUT_DIR, '3DImage.mha')
WriteImage(original_image, output_file_name_3D)

Select a specific DICOM series from a directory and only then load user selection.

In [ ]:
print_series_info <- function(series_ID, series_file_names) {
    img <- ReadImage(series_file_names[1])
    tags_to_print <- list('0010|0010' = 'Patient name: ', 
                          '0008|0060' = 'Modality: ',
                          '0008|0021' = 'Series date: ',
                          '0008|0080' = 'Institution name: ',
                          '0008|1050' = 'Performing physician\'s name: ')
    cat("Series ID: ", series_ID, "\n")
    invisible(lapply(names(tags_to_print), function(tag) { cat(tags_to_print[[tag]], suppressWarnings(img$GetMetaData(tag)),"\n")}))
    cat("\n")
}

data_directory <- dirname(fetch_data("CIRS057A_MR_CT_DICOM/readme.txt"))

series_IDs <- ImageSeriesReader_GetGDCMSeriesIDs(data_directory)
series_file_names <- lapply(series_IDs, function(series) {
                                        ImageSeriesReader_GetGDCMSeriesFileNames(data_directory, series)})
invisible(mapply(print_series_info, series_IDs, series_file_names))

In [ ]:
selected_series <- 1

img <- ReadImage(series_file_names[[selected_series]])
Show(img)

Write the volume as a series of JPEGs. The WriteImage function receives a volume and a list of images names and writes the volume according to the z axis. For a displayable result we need to rescale the image intensities (default is [0,255]) since the JPEG format requires a cast to the UInt8 pixel type.

In [ ]:
WriteImage(Cast(RescaleIntensity(img), "sitkUInt8"), 
           file.path(OUTPUT_DIR, sprintf("%03d.jpg", seq(1,img$GetDepth()))))

## Resampling

<img src="../figures/resampling.svg"/><br><br>

Resampling as the verb implies is the action of sampling an image, which itself is a sampling of an original continuous signal.

Generally speaking, resampling in SimpleITK involves four components:
1. Image - the image we resample, given in coordinate system $m$.
2. Resampling grid - a regular grid of points given in coordinate system $f$ which will be mapped to coordinate system $m$.
2. Transformation $T_f^m$ - maps points from coordinate system $f$ to coordinate system $m$, $^mp = T_f^m(^fp)$.
3. Interpolator - method for obtaining the intensity values at arbitrary points in coordinate system $m$ from the values of the points defined by the Image.


While SimpleITK provides a large number of interpolation methods, the two most commonly used are ```sitkLinear``` and ```sitkNearestNeighbor```. The former is used for most interpolation tasks, a compromise between accuracy and computational efficiency. The later is used to interpolate labeled images representing a segmentation, it is the only interpolation approach which will not introduce new labels into the result.

SimpleITK's procedural API provides three methods for performing resampling, with the difference being the way you specify the resampling grid:

1. ```Resample(const Image &image1, Transform transform, InterpolatorEnum interpolator, double defaultPixelValue, PixelIDValueEnum outputPixelType)```
2. ```Resample(const Image &image1, const Image &referenceImage, Transform transform, InterpolatorEnum interpolator, double defaultPixelValue, PixelIDValueEnum outputPixelType)```
3. ```Resample(const Image &image1, std::vector< uint32_t > size, Transform transform, InterpolatorEnum interpolator, std::vector< double > outputOrigin, std::vector< double > outputSpacing, std::vector< double > outputDirection, double defaultPixelValue, PixelIDValueEnum outputPixelType)```

In [ ]:
euler2d <- Euler2DTransform()
# Why do we set the center?
euler2d$SetCenter(logo$TransformContinuousIndexToPhysicalPoint(as.array(logo$GetSize())/2.0))

euler2d$SetTranslation(c(-16, -8))
euler2d$SetAngle(pi/10)

resampled_image <- Resample(logo, euler2d)
Show(resampled_image)

### Common Errors

It is not uncommon to end up with an empty (all black) image after resampling. This is due to:
1. Using wrong settings for the resampling grid, not too common, but does happen.
2. Using the inverse of the transformation $T_f^m$. This is a relatively common error, which is readily addressed by invoking the transformations ```GetInverse``` method.

### Defining the Resampling Grid

In the example above we arbitrarily used the original image grid as the resampling grid. As a result, for many of the transformations the resulting image contained black pixels, pixels which were mapped outside the spatial domain of the original image and a partial view of the original image.

If we want the resulting image to contain all of the original image no matter the transformation, we will need to define the resampling grid using our knowledge of the original image's spatial domain and the **inverse** of the given transformation. 

Computing the bounds of the resampling grid when dealing with an affine transformation is straightforward. An affine transformation preserves convexity with extreme points mapped to extreme points. Thus we only need to apply the **inverse** transformation to the corners of the original image to obtain the bounds of the resampling grid.

Computing the bounds of the resampling grid when dealing with a BSplineTransform or DisplacementFieldTransform is more involved as we are not guaranteed that extreme points are mapped to extreme points. This requires that we apply the **inverse** transformation to all points in the original image to obtain the bounds of the resampling grid.  

In [ ]:
euler2d <- Euler2DTransform()

euler2d$SetCenter(logo$TransformContinuousIndexToPhysicalPoint(as.array(logo$GetSize())/2.0))

tx <- 64
ty <- 32
euler2d$SetTranslation(c(tx, ty))

euler2d$SetAngle(0.79)

extreme_points <- list(logo$TransformIndexToPhysicalPoint(c(0,0)), 
                  logo$TransformIndexToPhysicalPoint(c(logo$GetWidth(),0)),
                  logo$TransformIndexToPhysicalPoint(c(logo$GetWidth(),logo$GetHeight())),
                  logo$TransformIndexToPhysicalPoint(c(0,logo$GetHeight())))
inv_euler2d <- euler2d$GetInverse()

extreme_points_transformed <- unlist(lapply(extreme_points, inv_euler2d$TransformPoint))
xs <- extreme_points_transformed[seq(1,length(extreme_points_transformed),2)]
ys <- extreme_points_transformed[seq(2,length(extreme_points_transformed),2)]
min_x <- min(xs)
min_y <- min(ys)
max_x <- max(xs)
max_y <- max(ys)

# Use the original spacing (arbitrary decision).
output_spacing <- logo$GetSpacing()
# Identity cosine matrix (arbitrary decision).   
output_direction <- c(1.0, 0.0, 0.0, 1.0)
# Minimal x,y coordinates are the new origin.
output_origin <- c(min_x, min_y)
# Compute grid size based on the physical size and spacing.
output_size <- c(as.integer((max_x-min_x)/output_spacing[1]), as.integer((max_y-min_y)/output_spacing[2]))

resampled_image <- Resample(logo, output_size, euler2d, "sitkLinear", output_origin, output_spacing, output_direction)
Show(resampled_image)

Are you puzzled by the result? Is the output just a copy of the input? Add a rotation to the code above and see what happens (```euler2d$SetAngle(0.79)```).

<a href="03_data_augmentation.ipynb"><h2 align=right>Next &raquo;</h2></a>